# Phase 2: Multi-Sailor Turn Order

5 sailors alternate turns: Alice → Bob → Charlie → Diana → Eve → Alice...

In [1]:
import sys
sys.path.insert(0, '../marooned_env')

from environment import MaroonedEnv
from config import ActionType, ResourceType, MapLevel, BASE_CAMP_POSITION
from models import Action, Position, InventoryItem

In [2]:
env = MaroonedEnv(seed=42)
observations = env.reset()

print(f"Sailors: {env.state.sailor_turn_order}")
print(f"Active: {env.state.get_active_sailor()}")

Sailors: ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']
Active: Alice


In [3]:
# Turn rotation test
for i in range(10):
    active = env.state.get_active_sailor()
    action = Action(sailor_id=active, action_type=ActionType.MOVE_NORTH)
    obs, reward, done, truncated, info = env.step({active: action})
    print(f"Turn {i+1}: {active} -> {env.state.get_active_sailor()}")

Turn 1: Alice -> Bob
Turn 2: Bob -> Charlie
Turn 3: Charlie -> Diana
Turn 4: Diana -> Eve
Turn 5: Eve -> Alice
Turn 6: Alice -> Bob
Turn 7: Bob -> Charlie
Turn 8: Charlie -> Diana
Turn 9: Diana -> Eve
Turn 10: Eve -> Alice


In [4]:
# Test visibility rules
alice = env.state.sailors["Alice"]
alice.backpack.append(InventoryItem(ResourceType.WOOD, 3))

alice_obs = env._generate_observation("Alice")
bob_obs = env._generate_observation("Bob")

print(f"Alice sees her backpack: {len(alice_obs.backpack)} items")
print(f"Bob sees his backpack: {len(bob_obs.backpack)} items")
print(f"Both see energy board: {alice_obs.all_sailors_energy == bob_obs.all_sailors_energy}")

Alice sees her backpack: 1 items
Bob sees his backpack: 0 items
Both see energy board: True


In [5]:
# Test 2-sailor ship building
env.state.common_inventory.clear()
env.state.common_inventory.append(InventoryItem(ResourceType.WOOD, 60))

base_pos = Position(*BASE_CAMP_POSITION)
env.state.sailors["Alice"].position = base_pos
env.state.sailors["Alice"].energy = 100

# 1 sailor - should fail
result = env._handle_build_ship("Alice")
print(f"1 sailor: {result['success']} - {result['reason']}")

# 2 sailors - should succeed
env.state.sailors["Bob"].position = base_pos
env.state.sailors["Bob"].energy = 100
result = env._handle_build_ship("Alice")
print(f"2 sailors: {result['success']} - Component: {result.get('component')}, Progress: {result.get('total_progress')}%")

1 sailor: False - Need at least 2 sailors at ship site (currently 1)
2 sailors: True - Component: hull, Progress: 30%
